(COMPC)=

# 5.3 Compilación a C

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

```{admonition} Al final de esta nota el y la lectora:
:class: tip

* 

```

Se presentan códigos y sus ejecuciones en una máquina `m4.16xlarge` de la nube de [AWS](https://aws.amazon.com/). Se utilizó la AMI `opt2-aws-educate-openblas-04-04-2021` de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

La máquina `m4.16xlarge` tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2290.913
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.03
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        8

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.11.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 256GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits
     *-bank:2
          description: DIMM RAM
          physical id: 2
          slot: DIMM 2
          size: 16GiB
          width: 64 bits
     *-bank:3
          description: DIMM RAM
          physical id: 3
          slot: DIMM 3
          size: 16GiB
          width: 64 bits
     *-bank:4
          description: DIMM RAM
          physical id: 4
          slot: DIMM

In [3]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-140 5.4.0-1038-aws #40-Ubuntu SMP Fri Feb 5 23:50:40 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con *import*'s. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

## Características de los lenguajes de programación

Los lenguajes de programación y sus implementaciones pueden tener características como las siguientes:

* Realizar un *parsing* de las instrucciones y ejecutar las acciones de forma casi inmediata (intérprete). Como ejemplo está [Beginners' All-purpose Symbolic Instruction Code: BASIC](https://en.wikipedia.org/wiki/BASIC)

* Realizar un *parsing* de las instrucciones, traducir a una [representación intermedia](https://en.wikipedia.org/wiki/Intermediate_representation) y ejecutar las acciones. Como ejemplo se encuentra *Python* en su implementación *CPython*. La interpretación de código intermedia es un [bytecode](https://en.wikipedia.org/wiki/Bytecode).

* Compilar [ahead of time](https://en.wikipedia.org/wiki/Ahead-of-time_compilation) las instrucciones antes de su ejecución. Como ejemplo se encuentran *C, C++* y *Fortran*.

* Realizar un *parsing* de las instrucciones y compilar las instrucciones en una forma [just in time compilation](https://en.wikipedia.org/wiki/Just-in-time_compilation) y *at* [runtime](https://en.wikipedia.org/wiki/Runtime_(program_lifecycle_phase)). Como ejemplos se encuentran *Julia* y *Python* en su implementación con [PyPy](https://en.wikipedia.org/wiki/PyPy).

La ejecución de instrucciones en los lenguajes de programación será más rápida dependiendo del lenguaje, la implementación que se utilice del mismo y de sus *features*.

## Python

In [1]:
%%file Rcf_python.py
import math
import time
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    sum_res = 0
    for i in range(n):
        x = a + (i + 1/2)*h_hat
        sum_res += f(x)
    return h_hat*sum_res

if __name__ == "__main__":   
    n = 10**7
    f = lambda x: math.exp(-x**2)
    a = 0
    b = 1
    start_time = time.time()
    res = Rcf(f,a,b,n)
    end_time = time.time()
    secs = end_time-start_time
    print("Rcf tomó", secs, "segundos" )

Writing Rcf_python.py


In [2]:
%%bash
python3 Rcf_python.py

Rcf tomó 3.4265081882476807 segundos


## R

In [6]:
%%file Rcf_R.R
library(tictoc)
Rcf<-function(f,a,b,n){
    '
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    '
    
    h_hat <- (b-a)/n
    sum_res <- 0
    for(i in 0:(n-1)){
        x <- a + (i + 1/2)*h_hat
        sum_res <- sum_res + f(x)
    }
    approx <- h_hat*sum_res
}
n <- 10**7
f <- function(x)exp(-x^2)
a <- 0
b <- 1
tic("Rcf tomó")
Rcf(f,a,b,n)
toc()

Writing Rcf_R.R


In [7]:
%%bash
Rscript Rcf_R.R

Rcf tomó: 5.568 sec elapsed


## Julia

In [8]:
%%file Rcf_julia.jl
"""
Compute numerical approximation using rectangle or mid-point
method in an interval.

# Arguments

- `f::Float`: function expression of integrand.
- `a::Integer`: left point of interval.
- `b::Integer`: right point of interval.
- `n::Integer`: number of subintervals.
"""
function Rcf(f, a, b, n)
    h_hat = (b-a)/n
    sum_res = 0
    for i in 0:n-1
        x = a + (i + 1/2)*h_hat
        sum_res += f(x)
    end    
    return h_hat*sum_res
end
function main()
    a = 0
    b = 1
    n =10^7
    f(x) = exp(-x^2)
    @time Rcf(f, a, b, n)
    @time Rcf(f, a, b, n)
end

main()

Writing Rcf_julia.jl


In [9]:
%%bash
/usr/local/julia-1.6.0/bin/julia Rcf_julia.jl

  0.228430 seconds
  0.228310 seconds


In [10]:
%%file Rcf_julia_typed_values.jl
"""
Compute numerical approximation using rectangle or mid-point
method in an interval.

# Arguments

- `f::Float`: function expression of integrand.
- `a::Integer`: left point of interval.
- `b::Integer`: right point of interval.
- `n::Integer`: number of subintervals.
"""
function Rcf(f, a, b, n)
    h_hat = (b-a)/n
    sum_res = 0.0
    for i in 0:n-1
        x = a + (i + 1/2)*h_hat
        sum_res += f(x)
    end    
    return h_hat*sum_res
end
function main()
    a = 0
    b = 1
    n =10^7
    f(x) = exp(-x^2)
    @time Rcf(f, a, b, n)
    @time Rcf(f, a, b, n)    
end

main()

Writing Rcf_julia_typed_values.jl


In [11]:
%%bash
/usr/local/julia-1.6.0/bin/julia Rcf_julia_typed_values.jl

  0.092623 seconds
  0.092402 seconds


## C

Para la medición de tiempos se utilizaron las ligas: [liga](https://stackoverflow.com/questions/16764276/measuring-time-in-millisecond-precision) y [liga2](https://www.techiedelight.com/find-execution-time-c-program/).

In [3]:
%%file Rcf_c.c
#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include<time.h>
#include <sys/time.h>

void Rcf(double ext_izq, double ext_der, int n,\
    double *suma_global_p);
double f(double nodo);
int main(int argc, char *argv[]){
    double suma_global = 0.0;
    double a = 0.0, b = 1.0;
    int n = 1e7;
    struct timeval start;
    struct timeval end;
    long seconds;
    long long mili;
    
    gettimeofday(&start, NULL);
    Rcf(a,b,n,&suma_global);
    gettimeofday(&end, NULL);
    seconds = (end.tv_sec - start.tv_sec);
    mili = 1000*(seconds) + (end.tv_usec - start.tv_usec)/1000;    
    printf("Tiempo de ejecución: %lld milisegundos", mili);
    
    return 0;
}
void Rcf(double a, double b, int n, double *sum){
    double h_hat = (b-a)/n;
    double x = 0.0;
    int i = 0;
    *sum = 0.0;
    for(i = 0; i <= n-1; i++){
        x = a + (i + 1/2.0)*h_hat;
        *sum += f(x);
    }
    *sum = h_hat*(*sum);
}
double f(double nodo){
    double valor_f;
    valor_f = exp(-pow(nodo,2));
    return valor_f;
}


Writing Rcf_c.c


In [4]:
%%bash
gcc -Wall Rcf_c.c -o Rcf_c.out -lm

In [5]:
%%bash
./Rcf_c.out

Tiempo de ejecución: 482 milisegundos

## [cython](https://cython.org/)

Cython es un compilador que traduce instrucciones escritas en Python "anotadas" en un módulo compilado que funciona como una extensión de Python. Los programas que se escriben con tales anotaciones resultan similares en sintaxis al lenguaje *C*. Este módulo puede ser importado como un módulo regular de Python utilizando `import`. 

```{margin}

La frase código tipo *CPU-bound* es código cuya ejecución involucra un porcentaje mayor para uso de CPU que uso de memoria o I/O.

```

*Cython* tiene un buen tiempo en la comunidad (2007 aproximadamente), es altamente usado y es de las herramientas preferidas para código tipo *CPU-bound*. 

Soporta la [API OpenMP](https://www.openmp.org/) para aprovechar los múltiples *cores* de una máquina.

Cython puede utilizarse vía un script `setup.py` que compila un módulo pero también puede utilizarse en `IPython` vía un comando `magic`.

## ¿En qué casos y qué tipo de ganancias en velocidad podemos esperar al usar Cython?

Un caso es en el que se tenga un código con muchos *loops* que realicen operaciones matemáticas típicamente no vectorizadas o que no pueden vectorizarse. Esto es, códigos en los que las instrucciones son básicamente sólo *Python* sin utilizar paquetes externos. Además, si en el ciclo las variables no cambian de su tipo (por ejemplo de `int` a `float`) entonces es un código que obtendrá ganancia en velocidad al compilar a código de máquina.

```{admonition} Observación
:class: tip

Si tu código de *Python* llama a operaciones vectorizadas vía *NumPy* podría ser que no se ejecute más rápido tu código después de compilarlo.

```

No esperamos tener un *speedup* después de compilar para llamadas a librerías externas (por ejemplo paqueterías que manejan bases de datos). También es poco probable que se obtengan ganancias significativas en programas que tengan alta carga de I/O.

En general es poco probable que tu código compilado se ejecute más rápido que un código en *C* "bien escrito" y también es poco probable que se ejecute más lento. Es muy posible que el código *C* generado desde *Python* mediante *Cython* pueda alcanzar las velocidades de un código escrito en *C*, a menos que la persona que programó en *C* tenga un gran conocimiento de formas de hacer que el código de *C* se ajuste a la arquitectura de la máquina sobre la que se ejecutan los códigos.

### Ejemplo con un script `setup.py`

In [1]:
import math
import time

from pytest import approx
from scipy.integrate import quad

Para este caso requerimos tres archivos:

1.El código que será compilado en un archivo con extensión `.pyx` (escrito en *Python*). 

```{admonition} Comentario

La extensión `.pyx` se utiliza en el lenguaje [Pyrex](https://www.csse.canterbury.ac.nz/greg.ewing/python/Pyrex/) que en términos simples es *Python* con manejo de tipo de valores de *C*. *Pyrex* traduce el código escrito en *Python* a código de *C* (por lo que ayuda a evitar el uso de la [Python/C API](https://docs.python.org/3/c-api/index.html) y permite la declaración de parámetros o valores en tipos de valores de *C*.

```

2.Un archivo `setup.py` que contiene las instrucciones para llamar a *cython* y cree el módulo compilado.

3.El código escrito en *Python* que importará el módulo compilado.

Archivo `.pyx`:

In [2]:
%%file Rcf_cython.pyx
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    nodes = [a+(i+1/2)*h_hat for i in range(n)]
    sum_res = 0
    for node in nodes:
        sum_res = sum_res+f(node)
    return h_hat*sum_res

Writing Rcf_cython.pyx


Archivo `setup.py` que contiene las instrucciones para el *build*:

In [3]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("Rcf_cython.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Writing setup.py


Compilar desde la línea de comandos:

In [4]:
%%bash
python3 setup.py build_ext --inplace

Compiling Rcf_cython.pyx because it changed.
[1/1] Cythonizing Rcf_cython.pyx
running build_ext
building 'Rcf_cython' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c Rcf_cython.c -o build/temp.linux-x86_64-3.7/Rcf_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.7/Rcf_cython.o -o /datos/MNO_desde_2018/ramas_repo/mno-master/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_cython.cpython-37m-x86_64-linux-gnu.so


Importar módulo compilado y ejecutarlo:

In [5]:
f=lambda x: math.exp(-x**2) #using math library

In [6]:
import Rcf_cython

In [7]:
n = 10**2
a = 0
b = 1
start_time = time.time()
res = Rcf_cython.Rcf(f, a, b,n)
end_time = time.time()

In [8]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.0001289844512939453 segundos


In [9]:
obj, err = quad(f, a, b)

In [10]:
print(res == approx(obj))

False


## Referencias de interés

* [Cython](https://en.wikipedia.org/wiki/Cython).

* [introduction to cython](http://okigiveup.net/an-introduction-to-cython/).

* [Basic cython tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html)

* [Source files and compilation](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html)

* [Compiling with a jupyter notebook](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiling-with-a-jupyter-notebook)

* [Dirk Eddelbuettel: rcpp ](http://dirk.eddelbuettel.com/code/rcpp.html).

* [Rcpp for everyone](https://teuder.github.io/rcpp4everyone_en/).

* [Introduction to rcpp:From Simple Examples to Machine Learning](http://dirk.eddelbuettel.com/papers/rcpp_rfinance_may2017.pdf).

* [Rcpp note](http://yixuan.cos.name/rcpp-note/index.html).

* [Rcpp quick reference guide](http://dirk.eddelbuettel.com/code/rcpp/Rcpp-quickref.pdf).

* [Learncpp](https://www.learncpp.com/).

* [Cplusplus](http://www.cplusplus.com/).

```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Referencias:**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

2. [H. Wickham, Advanced R, 2014](http://adv-r.had.co.nz/Rcpp.html)
